In [1]:
import sys
import os
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path + "/src/simulations_v2")
from load_params import load_multigroup_params, load_params
from multi_group_simulation import MultiGroupSimulation

In [2]:
base_params = load_params('../src/simulations_v2/params/quarantine_analysis/aug9quarantine/move_in_weekend/nominal_option_{}/ug_dorm.yaml'.format(1))[1]

In [3]:
from dynamic_population_sim import DynamicPopulationSim

movein_free_params = base_params.copy()
movein_selfiso_params = base_params.copy()
post_movein_params = base_params.copy()

daily_selfiso_to_free_contacts_pp = 1
daily_free_to_selfiso_contacts_pp = daily_selfiso_to_free_contacts_pp * \
                                    movein_selfiso_params['population_size'] / movein_free_params['population_size']

movein_contact_matrix = np.matrix(
    [[movein_free_params['expected_contacts_per_day'], daily_selfiso_to_free_contacts_pp],
    [daily_free_to_selfiso_contacts_pp, movein_selfiso_params['expected_contacts_per_day']]])

movein_time_horizon = 5

free_group_population_dynamics = {
    0: {'S': 4, 'pre_ID': 2},
    2: {'SyID_mild': 1},
    4: {'ID': 10}
}

selfiso_group_population_dynamics = {
    1: {'E': 50},
    2: {'QS': 5, 'QI': 1, 'R': 1, 'SyID_severe': 10}
}

dynamic_pop_sim = DynamicPopulationSim(
    movein_free_params,
    movein_selfiso_params,
    post_movein_params,
    movein_contact_matrix,
    movein_time_horizon,
    free_group_population_dynamics,
    selfiso_group_population_dynamics
)

In [4]:
sims = dynamic_pop_sim.movein_sim.sims

print("Initial population sizes: {} free, {} self-isolated".format(sims[0].pop_size, 
                                                                    sims[1].pop_size))
for t in range(movein_time_horizon):
    dynamic_pop_sim.step()
    print("After time {}, population sizes: {} free, {} self-isolated".format(t,
                                                                    sims[0].pop_size, 
                                                                    sims[1].pop_size))

print("At end of time {}, initialized post-movein simulation with population {}".format(t, dynamic_pop_sim.post_movein_sim.pop_size))

Initial population sizes: 4167 free, 4167 self-isolated
After time 0, population sizes: 4173 free, 4167 self-isolated
After time 1, population sizes: 4173 free, 4217 self-isolated
After time 2, population sizes: 4174 free, 4233 self-isolated
After time 3, population sizes: 4174 free, 4233 self-isolated
After time 4, population sizes: 4184 free, 4233 self-isolated
At end of time 4, initialized post-movein simulation with population 8417
